In [1]:
import pandapower.networks as pn
import pypower.api as pp
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

In [2]:
graphs = [pp.case300(), pp.case118(), pp.case57(), pp.case39()]

In [11]:
def analyse_graph(pp_graph, plot=True):
    node_ids = pp_graph['bus'][:,0].astype(np.int64)
    edges = pp_graph['branch'][:,0:2].astype(np.int64)

    G = nx.Graph()
    G.add_nodes_from(node_ids)
    for edge in edges:
        G.add_edge(edge[0], edge[1])

    degree_sequence = sorted((d for n, d in G.degree()), reverse=True)
    dmax = max(degree_sequence)

    if plot:
        fig = plt.figure("Degree of a random graph", figsize=(8, 8))
        # Create a gridspec for adding subplots of different sizes
        axgrid = fig.add_gridspec(5, 4)

        ax0 = fig.add_subplot(axgrid[0:3, :])
        Gcc = G.subgraph(sorted(nx.connected_components(G), key=len, reverse=True)[0])
        pos = nx.spring_layout(Gcc, seed=10396953)
        nx.draw_networkx_nodes(Gcc, pos, ax=ax0, node_size=20)
        nx.draw_networkx_edges(Gcc, pos, ax=ax0, alpha=0.4)
        ax0.set_title("Connected components of G")
        ax0.set_axis_off()

        ax1 = fig.add_subplot(axgrid[3:, :2])
        ax1.plot(degree_sequence, "b-", marker="o")
        ax1.set_title("Degree Rank Plot")
        ax1.set_ylabel("Degree")
        ax1.set_xlabel("Rank")

        ax2 = fig.add_subplot(axgrid[3:, 2:])
        ax2.bar(*np.unique(degree_sequence, return_counts=True))
        ax2.set_title("Degree histogram")
        ax2.set_xlabel("Degree")
        ax2.set_ylabel("# of Nodes")

        fig.tight_layout()
        plt.show()
    return np.unique(degree_sequence, return_counts=True)

# (array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 11]), array([69, 76, 84, 42, 14,  6,  5,  2,  1,  1], dtype=int64)) 

# (array([1, 2, 3, 4, 5, 6, 7, 8, 9]), array([ 7, 56, 19, 15, 11,  6,  2,  1,  1], dtype=int64)) 

# (array([1, 2, 3, 4, 5, 6]), array([ 1, 32, 12,  7,  3,  2], dtype=int64)) 

# (array([1, 2, 3, 4, 5]), array([ 9, 12, 14,  3,  1], dtype=int64)) 

In [17]:
def analyse_graphs(graphs):
    degree_frequencies = [analyse_graph(graph, plot=False) for graph in graphs]
    maxdegree = max([np.max(x[0]) for x in degree_frequencies])
    aggregated_degree_frequencies = {d: 0 for d in list(range(1, maxdegree+1))}
    for ds, cs in degree_frequencies:
        for d, c in zip(ds, cs):
            aggregated_degree_frequencies[d] += c
    frequencies = [x for x in aggregated_degree_frequencies.values()]
    # now we need to determine random degree sequences
    print(np.array(frequencies) / sum(frequencies))

        


analyse_graphs(graphs)

[0.16731518 0.34241245 0.25097276 0.13035019 0.05642023 0.02723735
 0.01361868 0.00583658 0.00389105 0.         0.00194553]
